In [ ]:
## このプログラムは「Mercari Price Suggestion Challenge」において
## 作成したノートブックで動作します

%%time
from datetime import datetime 
start_real = datetime.now()

import pandas as pd
# 訓練データテストデータをデータフレームに読み込む
train_df = pd.read_table('../input/mercari/train.tsv')
test_df = pd.read_table('../input/mercari/test.tsv')
print(train_df.shape, test_df.shape)

In [ ]:
# 3ドル未満のレコードをすべて削除する
train_df = train_df.drop(train_df[(train_df.price < 3.0)].index)
train_df.shape

In [ ]:
%%time
# 商品名と商品説明の単語の数を調べる

def wordCount(text):
    """
    Parameters:
      text(str): 商品名、商品の説明文
    """
    try:
        if text == 'No description yet':
            return 0  # 商品名や説明が'No description yet'の場合は0を返す
        else:
            text = text.lower()                  # すべて小文字にする
            words = [w for w in text.split(" ")] # スペースで切り分ける
            return len(words)                    # 単語の数を返す
    except: 
        return 0

# 'name'の各フィールドの単語数を'name_len'に登録
train_df['name_len'] = train_df['name'].apply(lambda x: wordCount(x))
test_df['name_len'] = test_df['name'].apply(lambda x: wordCount(x))
# 'item_description'の各フィールドの単語数を'desc_len'に登録
train_df['desc_len'] = train_df['item_description'].apply(lambda x: wordCount(x))
test_df['desc_len'] = test_df['item_description'].apply(lambda x: wordCount(x))

In [ ]:
%%time
# 訓練データのpriceのスケーリング
import numpy as np

# 訓練データの'price'を対数変換する
train_df["target"] = np.log1p(train_df.price)

In [ ]:
%%time
# カテゴリ名を切り分けて新設のカラムに登録する

def split_cat(text):
    """
    Parameters:
      text(str): カテゴリ名

    ・カテゴリを/で切り分ける
    ・データが存在しない場合は"No Label"を返す
    """
    try: return text.split("/")
    except: return ("No Label", "No Label", "No Label")

# 3つに切り分けたカテゴリ名を'subcat_0'、'subcat_1'、'subcat_2'に登録
# 訓練データ
train_df['subcat_0'], train_df['subcat_1'], train_df['subcat_2'] = \
    zip(*train_df['category_name'].apply(lambda x: split_cat(x)))
# テストデータ
test_df['subcat_0'], test_df['subcat_1'], test_df['subcat_2'] = \
    zip(*test_df['category_name'].apply(lambda x: split_cat(x)))

In [ ]:
%%time
# 'brand_name'の対策

# train_dfとtest_dfを縦方向に結合
full_set = pd.concat([train_df, test_df])
# full_setの'brand_name'から重複なしのブランドリスト(集合)を生成
all_brands = set(full_set['brand_name'].values)

# 'brand_name'の欠損値NaNを'missing'に置き換える
train_df['brand_name'].fillna(value='missing', inplace=True)
test_df['brand_name'].fillna(value='missing', inplace=True)

# 訓練データの'brand_name'が'missing'に一致するレコード数を取得
train_premissing = len(train_df.loc[train_df['brand_name'] == 'missing'])
# テストデータの'brand_name'が'missing'に一致するレコード数を取得
test_premissing = len(test_df.loc[test_df['brand_name'] == 'missing'])

def brandfinder(line):
    """
    Parameters: line(str): ブランド名

    ・ブランド名の'missing'を商品名に置き換える:
         missing'の商品名の単語がブランドリストに存在する場合
    ・ブランド名を商品名に置き換える:
        商品名がブランドリストの名前と完全に一致する場合
    ・ブランド名をそのままにする:
        商品名がブランドリストの名前と一致しない
        ブランド名が'missing'だが商品名の単語がブランドリストにない
    """
    brand = line[0] # 第1要素はブランド名
    name = line[1]  # 第2要素は商品名
    namesplit = name.split(' ') # 商品名をスペースで切り分ける
    
    if brand == 'missing':  # ブランド名が'missing'と一致
        for x in namesplit: # 商品名から切り分けた単語を取り出す
            if x in all_brands:                
                return name # 単語がブランドリストに一致したら商品名を返す
    if name in all_brands:  # 商品名がブランドリストに存在すれば商品名を返す
        return name
    
    return brand            # どれにも一致しなければブランド名を返す

# ブランド名の付替えを実施
train_df['brand_name'] = train_df[['brand_name','name']].apply(brandfinder, axis = 1)
test_df['brand_name'] = test_df[['brand_name','name']].apply(brandfinder, axis = 1)

# 書き換えられた'missing'の数を取得
train_found = train_premissing-len(train_df.loc[train_df['brand_name'] == 'missing'])
test_found = test_premissing-len(test_df.loc[test_df['brand_name'] == 'missing'])
print(train_premissing) # 書き換える前の'missing'の数
print(train_found)      # 書き換えられた'missing'の数
print(test_premissing)  # 書き換える前の'missing'の数
print(test_found)       # 書き換えられた'missing'の数

In [ ]:
%%time
# 訓練用のデータフレームを訓練用と検証用に99:1で分割する
from sklearn.model_selection import train_test_split
import gc

train_dfs, dev_dfs = train_test_split(
    train_df,         # 対象のデータフレーム
    random_state=123, # 乱数生成時のシード(種)
    train_size=0.99,  # 訓練用に99%のデータ
    test_size=0.01)   # 検証用に1%のデータ

n_trains = train_dfs.shape[0] # 訓練データのサイズ
n_devs = dev_dfs.shape[0]     # 検証データのサイズ
n_tests = test_df.shape[0]   # テストデータのサイズ
print('Training :', n_trains, 'examples')
print('Validating :', n_devs, 'examples')
print('Testing :', n_tests, 'examples')

del train_df
gc.collect()

In [ ]:
%%time
# 訓練データ、検証データ、テストデータを1つのデータフレームに連結
full_df = pd.concat([train_dfs, dev_dfs, test_df])

def fill_missing_values(df):
    """連結データのカテゴリ名、ブランド名、説明文のNaNを'missing'に置き換える
    
    Parameter:
        df: すべてのデータを連結したデータフレーム
    """
    df.category_name.fillna(value='missing', inplace=True)    # カテゴリ名
    df.brand_name.fillna(value='missing', inplace=True)       # ブランド名
    df.item_description.fillna(value='missing', inplace=True) # 説明文
    # 説明文の'No description yet'を'missing'にする
    df.item_description.replace(
        'No description yet','missing', inplace=True) # 説明文の置き換え
    return df

full_df = fill_missing_values(full_df)

In [ ]:
%%time

# カテゴリ、ブランド、3カテゴリのテキストをラベルエンコードする
from sklearn.preprocessing import LabelEncoder

print("Processing categorical data...")

# LabelEncoderの生成
le = LabelEncoder()

# 'category_name'をエンコードして'category'カラムに登録する
le.fit(full_df.category_name)
full_df['category'] = le.transform(full_df.category_name)

# 'brand_name'のエンコード
le.fit(full_df.brand_name)
full_df.brand_name = le.transform(full_df.brand_name)

# 'subcat_0'のエンコード
le.fit(full_df.subcat_0)
full_df.subcat_0 = le.transform(full_df.subcat_0)

# 'subcat_1'のエンコード
le.fit(full_df.subcat_1)
full_df.subcat_1 = le.transform(full_df.subcat_1)

# 'subcat_2'のエンコード
le.fit(full_df.subcat_2)
full_df.subcat_2 = le.transform(full_df.subcat_2)

del le
gc.collect()

In [ ]:
%%time
# 連結データの説明文、商品名をTokenizerでラベルエンコードする
from tensorflow.keras.preprocessing.text import Tokenizer

# 説明文、商品名、カテゴリ名の配列要素を以下のように1次元配列に連結する
# [[説明1,説明2, ..., 商品1,商品2, ..., カテゴリ1,カテゴリ2, ...]
#
print("Transforming text data to sequences...")
raw_text = np.hstack(
    [full_df.item_description.str.lower(), # 説明文
     full_df.name.str.lower(),             # 商品名
     full_df.category_name.str.lower()]    # カテゴリ名
)
print('sequences shape', raw_text.shape)

print("   Fitting tokenizer...")
tok_raw = Tokenizer()
tok_raw.fit_on_texts(raw_text)

print("   Transforming text to sequences...")
full_df['seq_item_description'] = tok_raw.texts_to_sequences(full_df.item_description.str.lower())
full_df['seq_name'] = tok_raw.texts_to_sequences(full_df.name.str.lower())

del tok_raw
gc.collect()

In [ ]:
# RNNモデルで使用する定数を定義

MAX_NAME_SEQ = 10      # 商品名の最大サイズ(最大17を10に切り詰める)
MAX_ITEM_DESC_SEQ = 75 # 商品説明の最大サイズ(最大269を75に切り詰める)
MAX_CATEGORY_SEQ = 8   # カテゴリ名の最大サイズ(最大8)

# 商品名ラベルと商品説明の単語数: ラベルの最大値+100
MAX_TEXT = np.max([
    np.max(full_df.seq_name.max()),
    np.max(full_df.seq_item_description.max())
]) + 100
# カテゴリ名の単語数: カテゴリラベルの最大値+1
MAX_CATEGORY = np.max(full_df.category.max()) + 1
# ブランド名の単語数: ブランドラベルの最大値+1
MAX_BRAND = np.max(full_df.brand_name.max()) + 1
# 商品状態の数: 商品状態の最大値+1
MAX_CONDITION = np.max(full_df.item_condition_id.max()) + 1
# 商品説明の単語数: レコードごとの単語数の最大値+1
MAX_DESC_LEN = np.max(full_df.desc_len.max()) + 1
# 商品名の単語数: レコードごとの単語数の最大値+1
MAX_NAME_LEN = np.max(full_df.name_len.max()) + 1
# サブカテゴリの単語数: 各ラベルの最大値+1
MAX_SUBCAT_0 = np.max(full_df.subcat_0.max()) + 1
MAX_SUBCAT_1 = np.max(full_df.subcat_1.max()) + 1
MAX_SUBCAT_2 = np.max(full_df.subcat_2.max()) + 1

In [ ]:
%%time

# RNNに入力するデータを用意する
from tensorflow.keras.preprocessing.sequence import pad_sequences

def get_rnn_data(dataset):
    """
    入力データをdictオブジェクトに格納して返す
    
    Parameter: 
      dataset: フル結合のデータフレーム
    """
    X = {
        # 商品名ラベル(ndarray(int))
        # 0パディングして配列のサイズを統一: MAX_NAME_SEQ=10
        'name': pad_sequences(dataset.seq_name,
                              maxlen=MAX_NAME_SEQ),
        # 商品説明ラベル(ndarray(int))
        # 0パディングして配列のサイズを統一: MAX_ITEM_DESC_SEQ=75
        'item_desc': pad_sequences(dataset.seq_item_description,
                                   maxlen=MAX_ITEM_DESC_SEQ),
        # ブランド名のラベル(ndarray(int))
        'brand_name': np.array(dataset.brand_name),
        # /区切りのカテゴリ名のラベル(ndarray(int))
        'category': np.array(dataset.category),
        # 商品の状態(ndarray(int))
        'item_condition': np.array(dataset.item_condition_id),
        # 送料負担(ndarray(int)):出品者負担は1,購入者負担は0
        'num_vars': np.array(dataset[["shipping"]]),
        # 商品説明の単語数(ndarray(int))
        'desc_len': np.array(dataset[["desc_len"]]),
        # 商品名の単語数(ndarray(int))
        'name_len': np.array(dataset[["name_len"]]),
        # カテゴリ0のラベル(ndarray(int))
        'subcat_0': np.array(dataset.subcat_0),
        # カテゴリ1のラベル(ndarray(int))
        'subcat_1': np.array(dataset.subcat_1),
        # カテゴリ2のラベル(ndarray(int))
        'subcat_2': np.array(dataset.subcat_2),
    }
    return X

# フル結合のデータフレームから抽出
# 訓練データ: インデックス0～訓練データ数-1のインデックスまで
train = full_df[:n_trains]
# 検証データ: 訓練データ数～訓練データ数+検証データ数-1のインデックスまで
dev = full_df[n_trains:n_trains+n_devs]
# テストデータ: 訓練データ+検証データから末尾まで
test = full_df[n_trains+n_devs:]

# 訓練用のdictを取得
X_train = get_rnn_data(train)
# 訓練用の商品価格の1階テンソルを2階テンソルに変換
# (14817,)→(14817,1)
Y_train = train.target.values.reshape(-1, 1)

# 検証用のdictを取得
X_dev = get_rnn_data(dev)
# 検証用の商品価格の1階テンソルを2階テンソルい変換
# (146844,)→(146844,1)
Y_dev = dev.target.values.reshape(-1, 1)

# テスト用のdictを取得
X_test = get_rnn_data(test)

del full_df
gc.collect()

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense, Embedding, Flatten
from tensorflow.keras.layers import concatenate, GRU
from tensorflow.keras.optimizers import Adam

np.random.seed(123) # 乱数のシードを設定

# RMSE(Root Mean Square Error):二乗平均平方根誤差
#
# 予測をチェックするために使用
# この関数を使用する際のYとY_predはすでにlogスケールになっているので、RMSLE(対数二乗平均平方根誤差)のように機能する
def rmsle(Y, Y_pred):
    assert Y.shape == Y_pred.shape
    return np.sqrt(np.mean(np.square(Y_pred - Y )))

def new_rnn_model(lr=0.001, decay=0.0):
    """RNNのモデルを生成
    
    Parameters:
      lr: 学習率
      decay: 学習率減衰
    """
    # 入力層
    # 商品名ラベル、商品説明ラベル、ブランド名ラベル、商品の状態、送料負担、
    name = Input(shape=[X_train["name"].shape[1]], name="name")
    item_desc = Input(shape=[X_train["item_desc"].shape[1]], name="item_desc")
    brand_name = Input(shape=[1], name="brand_name")
    item_condition = Input(shape=[1], name="item_condition")
    num_vars = Input(shape=[X_train["num_vars"].shape[1]], name="num_vars")
    # 商品名テキスト、商品説明テキストの単語数
    name_len = Input(shape=[1], name="name_len")
    desc_len = Input(shape=[1], name="desc_len")
    # サブカテゴリラベル0～2
    subcat_0 = Input(shape=[1], name="subcat_0")
    subcat_1 = Input(shape=[1], name="subcat_1")
    subcat_2 = Input(shape=[1], name="subcat_2")

    # Embedding層
    # 商品名のEmbedding: 入力は単語の総数+100、出力の次元数は20
    emb_name = Embedding(MAX_TEXT, 20)(name)
    # 商品説明のEmbedding: 入力は単語の総数+100、出力の次元数は60
    emb_item_desc = Embedding(MAX_TEXT, 60)(item_desc)
    # ブランド名のEmbedding: 入力は単語の総数+1、出力の次元数は10
    emb_brand_name = Embedding(MAX_BRAND, 10)(brand_name)
    # 商品状態のEmbedding: 入力は5+1、出力の次元数は5
    emb_item_condition = Embedding(MAX_CONDITION, 5)(item_condition)
    # 商品説明の単語数のEmbedding: 入力は商品説明の最大単語数+1、出力は5
    emb_desc_len = Embedding(MAX_DESC_LEN, 5)(desc_len)
    # 商品名の単語数のEmbedding: 入力は商品名の最大単語数+1、出力は5
    emb_name_len = Embedding(MAX_NAME_LEN, 5)(name_len)
    # サブカテゴリ0～2のEmbedding: 入力はカテゴリ名の最大単語数+1、出力は10
    emb_subcat_0 = Embedding(MAX_SUBCAT_0, 10)(subcat_0)
    emb_subcat_1 = Embedding(MAX_SUBCAT_1, 10)(subcat_1)
    emb_subcat_2 = Embedding(MAX_SUBCAT_2, 10)(subcat_2)
    
    # ReccuurentユニットはLSTMより高速なGRU
    rnn_layer1 = GRU(16) (emb_item_desc) # 商品説明のユニット
    rnn_layer2 = GRU(8) (emb_name)       # 商品名のユニット

    # 全結合層
    main_l = concatenate([
        Flatten()(emb_brand_name),     # ブランド名のEmbedding
        Flatten()(emb_item_condition), # 商品状態のEmbedding
        Flatten()(emb_desc_len), # 商品説明の単語数のEmbedding
        Flatten()(emb_name_len), # 商品名の単語数のEmbedding
        Flatten()(emb_subcat_0), # サブカテゴリ0のEmbedding
        Flatten()(emb_subcat_1), # サブカテゴリ1のEmbedding
        Flatten()(emb_subcat_2), # サブカテゴリ2のEmbedding
        rnn_layer1, # 商品説明のGRUユニット
        rnn_layer2, # 商品名のGRUユニット
        num_vars    # 送料負担(0か1)
    ])
    # 512、256、128、64ユニットの層を追加
    main_l = Dropout(0.1)(
        Dense(512,kernel_initializer='normal',activation='relu')(main_l))
    main_l = Dropout(0.1)(
        Dense(256,kernel_initializer='normal',activation='relu')(main_l))
    main_l = Dropout(0.1)(
        Dense(128,kernel_initializer='normal',activation='relu')(main_l))
    main_l = Dropout(0.1)(
        Dense(64,kernel_initializer='normal',activation='relu')(main_l))

    # 出力層(1ユニット)
    output = Dense(1, activation="linear") (main_l)
    
    # Modelオブジェクトの生成
    model = Model(
        # 入力層はマルチ入力モデルなのでリストにする
        inputs=[name, item_desc, brand_name, item_condition, num_vars,
                desc_len, name_len, subcat_0, subcat_1, subcat_2],
        # 出力層
        outputs=output
    )

    # 平均2乗誤差関数とオプティマイザーをセットしてコンパイル
    model.compile(loss = 'mse',
                  optimizer = Adam(lr=lr, decay=decay))

    return model

# RNNのモデルを生成
model = new_rnn_model()
model.summary()
del model
gc.collect()

In [ ]:
%%time

# ミニバッチのサイズ
BATCH_SIZE = 512 * 2
epochs = 3

# 学習率減衰
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
steps = int(len(X_train['name']) / BATCH_SIZE) * epochs
lr_init =  0.005
lr_fin = 0.001
lr_decay = exp_decay(lr_init, lr_fin, steps)

# モデルを生成
rnn_model = new_rnn_model(lr=lr_init, decay=lr_decay)
# 学習
rnn_model.fit(X_train, Y_train,
              epochs=epochs,
              batch_size=BATCH_SIZE,
              validation_data=(X_dev, Y_dev),
              verbose=1
)

In [ ]:
%%time
# 検証データでモデルを評価する

print("Evaluating the model on validation data...")
# 学習済みのモデルで検証データの予測を行う
Y_dev_preds_rnn = rnn_model.predict(X_dev,
                                    batch_size=BATCH_SIZE)
# 予測値の損失をrmsle()で求める
print(" RMSLE error:", rmsle(Y_dev, # 検証データの商品価格
                             Y_dev_preds_rnn))

In [ ]:
# テストデータで予測する
rnn_preds = rnn_model.predict(X_test, batch_size=BATCH_SIZE, verbose=1)
# 指数関数
rnn_preds = np.expm1(rnn_preds)

del rnn_model
gc.collect()

In [ ]:
# Ridgeモデルのための前処理

# 訓練データ、検証データ、テストデータを結合する
full_df2 = pd.concat([train_dfs, dev_dfs, test_df])

In [ ]:
%%time

# 欠損値を処理し、すべてのデータを文字列にする
print("Handling missing values...")
# カテゴリ名の欠損値を'missing'に置き換える
full_df2['category_name'] = \
    full_df2['category_name'].fillna('missing').astype(str)
# サブカテゴリのラベルを文字列に変換
full_df2['subcat_0'] = full_df2['subcat_0'].astype(str)
full_df2['subcat_1'] = full_df2['subcat_1'].astype(str)
full_df2['subcat_2'] = full_df2['subcat_2'].astype(str)
# ブランド名の欠損値を'missing'に置き換える
full_df2['brand_name'] = full_df2['brand_name'].fillna('missing').astype(str)
# 送料負担、商品の状態を文字列に置き換える
full_df2['shipping'] = full_df2['shipping'].astype(str)
full_df2['item_condition_id'] = full_df2['item_condition_id'].astype(str)
# 説明文の単語数、商品名の単語数を文字列に置き換える
full_df2['desc_len'] = full_df2['desc_len'].astype(str)
full_df2['name_len'] = full_df2['name_len'].astype(str)
# 説明文の欠損値を'No description yet'に置き換える
full_df2['item_description'] = \
    full_df2['item_description'].fillna('No description yet').astype(str)

In [ ]:
%%time

# すべてのデータをBag of Wordsでベクトル化する
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion

print("Vectorizing data...")
# CountVectorizerの生成処理を関数化
default_preprocessor = CountVectorizer().build_preprocessor()

def build_preprocessor(field):
    """
    指定されたカラムのインデックスを取得し、
    トークンカウント行列を作成するためのCountVectorizerを返す
    
    Parameter:field(str)
      フル結合データフレームのカラム名
    """
    field_idx = list(full_df2.columns).index(field)
    return lambda x: default_preprocessor(x[field_idx])

# トークンカウント行列
# CountVectorizeを結合して
# (識別子, ベクトライザーオブジェクト)のリストで構成される
# トランスファーマーオブジェクトを生成
vectorizer = FeatureUnion([
    ('name', CountVectorizer(
        # bag-of-wordで分割する単位をn-gramで連続する単語のつながりとする
        # 商品名は2-gram(2つの単語のつながり)で分割
        ngram_range=(1, 2),
        max_features=5000, # トークンカウントの上限値############### 50000
        # トークンカウントステップをオーバーライド
        preprocessor=build_preprocessor('name'))),
    ('subcat_0', CountVectorizer(
        # トークンの構成を示す正規表現を'+'として1文字に対応
        token_pattern='.+',
        preprocessor=build_preprocessor('subcat_0'))),
    ('subcat_1', CountVectorizer(
        token_pattern='.+',
        preprocessor=build_preprocessor('subcat_1'))),
    ('subcat_2', CountVectorizer(
        token_pattern='.+',
        preprocessor=build_preprocessor('subcat_2'))),
    ('brand_name', CountVectorizer(
        token_pattern='.+',
        preprocessor=build_preprocessor('brand_name'))),
    ('shipping', CountVectorizer(
        token_pattern='\d+',
        preprocessor=build_preprocessor('shipping'))),
    ('item_condition_id', CountVectorizer(
        token_pattern='\d+',
        preprocessor=build_preprocessor('item_condition_id'))),
    ('desc_len', CountVectorizer(
        token_pattern='\d+',
        preprocessor=build_preprocessor('desc_len'))),
    ('name_len', CountVectorizer(
        token_pattern='\d+',
        preprocessor=build_preprocessor('name_len'))),
    ('item_description', TfidfVectorizer(
        # bag-of-wordで分割する単位をn-gramで連続する単語のつながりとする
        # 商品説名は3-gram(3つの単語のつながり)で分割
        ngram_range=(1, 3),
        max_features=5000, # トークンカウントの上限値#################### 100000
        preprocessor=build_preprocessor('item_description'))),
])

# フル結合のデータフレームのフィールド値を
# n-grainによるbag-of-wordsでトークンカウント行列に変換する
X = vectorizer.fit_transform(full_df2.values)

del vectorizer
gc.collect()

# 入力用のdictオブジェクトから訓練用のデータを抽出
X_train = X[:n_trains]
# 訓練用のデータフレームから商品価格を抽出して
# (データ数, 価格)の2階テンソルに変換
Y_train = train_dfs.target.values.reshape(-1, 1)

# 入力用のdictオブジェクトから検証用のデータを抽出
X_dev = X[n_trains:n_trains+n_devs]
# 検証用のデータフレームから商品価格を抽出して
# (データ数, 価格)の2階テンソルに変換
Y_dev = dev_dfs.target.values.reshape(-1, 1)

# 入力用のdictオブジェクトからテストデータを抽出
X_test = X[n_trains+n_devs:]

print('X:', X.shape)
print('X_train:', X_train.shape)
print('X_dev:', X_dev.shape)
print('X_test:', X_test.shape)
print('Y_train:', Y_train.shape)
print('Y_dev:', Y_dev.shape)

In [ ]:
%%time
# Ridge、RidgeCVで最適化する

from sklearn.linear_model import Ridge, RidgeCV

print("Fitting Ridge model on training examples...")
ridge_model = Ridge(
    solver='auto',      # ソルバーをオートモードにする
    fit_intercept=True, # 切片を計算に使用
    alpha=1.0,          # 正則化の強度はデフォルト値
    max_iter=200,       # ソルバーの最大反復回数
    normalize=False,    # 正規化を行う
    tol=0.01,           # 回帰の反復を停止するときの精度
    # データをシャッフルするときに使用する疑似乱数ジェネレータのシード
    random_state = 1,
)

print("Fitting RidgeCV model on training examples...")
ridge_modelCV = RidgeCV(
    fit_intercept=True, # 切片を計算に使用
    alphas=[5.0],
    normalize=False,
    cv = 2, # 交差検証時にスコアを2回連続して(毎回異なる分割で)計算
    # モデルの評価は平均2乗誤差回帰損失で行う
    scoring='neg_mean_squared_error',
)

ridge_model.fit(X_train, Y_train)
ridge_modelCV.fit(X_train, Y_train)

In [ ]:
# Ridgeモデルに検証データを入力して損失を測定
Y_dev_preds_ridge = ridge_model.predict(X_dev)
Y_dev_preds_ridge = Y_dev_preds_ridge.reshape(-1, 1)
print('Ridge model RMSL error:', rmsle(Y_dev, Y_dev_preds_ridge))

In [ ]:
# RidgeCVモデルに検証データを入力して損失を測定
Y_dev_preds_ridgeCV = ridge_modelCV.predict(X_dev)
Y_dev_preds_ridgeCV = Y_dev_preds_ridgeCV.reshape(-1, 1)
print('RidgeCV model RMSL error:', rmsle(Y_dev, Y_dev_preds_ridgeCV))

In [ ]:
%%time
# テストデータで予測する

# Ridgeモデル
ridge_preds = ridge_model.predict(X_test)
ridge_preds = np.expm1(ridge_preds)

# RidgeCVモデル
ridgeCV_preds = ridge_modelCV.predict(X_test)
ridgeCV_preds = np.expm1(ridgeCV_preds)

In [ ]:
%%time
def aggregate_predicts3(Y1, Y2, Y3, ratio1, ratio2):
    """3モデルの予測値に重みを適用し、3つの予測値を1つに結合して返す
    Y1: RNNの予測値
    Y2: Ridgeの予測値
    Y3: Ridge2の予測値
    ratio1: 重み1
    ratio2: 重み2
    """
    assert Y1.shape == Y2.shape
    return Y1*ratio1 + Y2*ratio2 + Y3*(1.0 - ratio1-ratio2)

In [ ]:
%%time
# 3モデルのアンサンブル
# 実行結果は書籍に掲載のものと一致するものではありません

best1 = 0
best2 = 0
lowest = 0.99
for i in range(100):
    for j in range(100):
        r = i*0.01
        r2 = j*0.01
        # r+r2が1.0以下なら
        if r+r2 < 1.0:
            # 3モデルの予測値に重みを適用して新たな予測値を取得
            Y_dev_preds = aggregate_predicts3(
                Y_dev_preds_rnn,     # RNNの検証データ予測
                Y_dev_preds_ridgeCV, # Ridgeの検証データ予測
                Y_dev_preds_ridge,   # RidgeCVの検証データ予測
                r, r2)               # 増加中の重み
            # 現在の重みを適用後の予測値と検証データの正解値との損失を求める
            fpred = rmsle(Y_dev, Y_dev_preds)
            # 現在の損失が小さければ重みの比率を記録する
            if fpred < lowest:
                best1 = r
                best2 = r2
                # 現在のベストな損失として記録する
                lowest = fpred
            #print(str(r)+"-RMSL error for RNN + Ridge + RidgeCV on dev set:", fpred)

# 3モデルのアンサンブルによる検証データの予測
Y_dev_preds = aggregate_predicts3(
    Y_dev_preds_rnn,
    Y_dev_preds_ridgeCV,
    Y_dev_preds_ridge, best1, best2)

print('r1:', best1)
print('r2:', best2)
print('r3:', 1.0-best1-best2)
print("(Best) RMSL error for RNN + Ridge + RidgeCV on dev set:\n",
      rmsle(Y_dev, Y_dev_preds)) # Y_dev_predsのRMSLでの損失を調べる

In [ ]:
# 3モデルのアンサンブルで予測する
preds = aggregate_predicts3(rnn_preds,     # RNNのテストデータの予測
                            ridgeCV_preds, # Ridgeのテストデータの予測
                            ridge_preds,   # RidgeCVのテストデータの予測
                            best1, best2)  # ベストな重み
submission = pd.DataFrame({
        "test_id": test_df.test_id,
        "price": preds.reshape(-1),
})
submission.to_csv("./rnn_ridge_submission_best.csv", index=False)

In [ ]:
stop_real = datetime.now()
execution_time_real = stop_real-start_real 
print(execution_time_real)